In [2]:
import pennylane as qml
import numpy as np
import torch
from torch import nn
import scipy.io as sio

In [3]:
# Global parameters
ENVIR = 'indoor'
LAYERS = 2
BATCHES = 5
CHANNELS = 2

# image parameters
IMG_HEIGHT = 32
IMG_WIDTH = IMG_HEIGHT
IMG_DIM = IMG_HEIGHT * IMG_WIDTH * CHANNELS
IMG_QUBITS = int(np.log2(IMG_DIM))

# compressed parameters
COM_HEIGHT = 16
COM_WIDTH = COM_HEIGHT 
COM_DIM = COM_HEIGHT * COM_WIDTH * CHANNELS
COM_QUBITS = int(np.log2(COM_DIM))

ALL_QUBITS = IMG_QUBITS + 1
ANC_QUBITS = IMG_QUBITS - COM_QUBITS

In [4]:
class ClassicalNN(nn.Module):
    ''' 构造经典压缩神经网络 '''
    def __init__(self, channels, img_height, com_height):
        super().__init__()
        self.img_dim = channels * img_height**2
        self.com_dim = channels * com_height**2
        self.conv = nn.Conv2d(in_channels=channels, out_channels=2, kernel_size=3, stride=1, padding=1, bias=True)
        self.leaky_relu = nn.LeakyReLU(negative_slope=0.2)
        self.dense_encode = nn.Linear(in_features=self.img_dim, out_features=self.com_dim)
        self.bn1d = nn.BatchNorm1d(num_features=channels)

    def forward(self, x):
        ''' 定义经典压缩层 '''
        x = self.conv(x)
        x = self.leaky_relu(x)
        x = x.reshape((x.shape[0], 1, self.img_dim))
        x = self.dense_encode(x)
        x = 2 * self.bn1d(x)  # 注意在这里完成了乘2
        return x


def frqi_encoder(qubits, params, target=0):
    ''' construct the FRQI encoding circuit '''
    for index in range(2**qubits):
        binary_str = bin(index)[2:].zfill(qubits)
        bits = [int(bit) for bit in binary_str]
        bits.reverse()  # 原地逆序，高位在右，作用于序数大的比特位
        qml.ctrl(qml.RY, control=range(1, qubits + 1), control_values=bits)(params[index], wires=target)

In [6]:
coe = [-1]
obs_list = [qml.PauliZ(0)]
hamiltonian = qml.Hamiltonian(coe, observables=obs_list)

In [7]:
dev = qml.device('default.qubit', wires=ALL_QUBITS)


@qml.qnode(dev, interface='torch')
def frqi_circuit(com_params, img_params, asz_params):
    ''' construct the complete quantum circuit '''
    for i in range(1, COM_QUBITS + 1):
        qml.Hadamard(wires=i)
    frqi_encoder(COM_QUBITS, com_params)
    qml.StronglyEntanglingLayers(weights=asz_params, wires=range(ALL_QUBITS))
    frqi_encoder(IMG_QUBITS, img_params)
    return qml.expval(hamiltonian)

In [19]:
class HybridNN(nn.Module):
    ''' 把上面定义的经典神经网络和量子神经网络组装成完整神经网络 '''
    def __init__(self, classical_nn):
        super().__init__()
        self.classical_nn = classical_nn
        self.asz_params = np.random.uniform(0, np.pi, size=(LAYERS, ALL_QUBITS, 3))

    def forward(self, x):
        ''' 在量子线路前，加上经典压缩网络 '''
        com_params = self.classical_nn(x)
        reverse_x = (-1) * x
        loss = frqi_circuit(com_params, reverse_x, self.asz_params)
        return loss

In [9]:
# Data loading
if ENVIR == 'indoor':
    mat = sio.loadmat('../../DataSpace/csinet/data/DATA_Htrainin.mat')
    x_train = mat['HT']  # array
    mat = sio.loadmat('../../DataSpace/csinet/data/DATA_Hvalin.mat')
    x_val = mat['HT']  # array
    mat = sio.loadmat('../../DataSpace/csinet/data/DATA_Htestin.mat')
    x_test = mat['HT']  # array

elif ENVIR == 'outdoor':
    mat = sio.loadmat('../../DataSpace/csinet/data/DATA_Htrainout.mat')
    x_train = mat['HT']  # array
    mat = sio.loadmat('../../DataSpace/csinet/data/DATA_Hvalout.mat')
    x_val = mat['HT']  # array
    mat = sio.loadmat('../../DataSpace/csinet/data/DATA_Htestout.mat')
    x_test = mat['HT']  # array

In [10]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
print('x_train 的原始维度:', x_train.shape)

x_train = np.reshape(x_train, (len(x_train), CHANNELS, IMG_HEIGHT, IMG_WIDTH))
x_val = np.reshape(x_val, (len(x_val), CHANNELS, IMG_HEIGHT, IMG_WIDTH))
x_test = np.reshape(x_test, (len(x_test), CHANNELS, IMG_HEIGHT, IMG_WIDTH))
print('x_train 的塑形维度:', x_train.shape)

x_train 的原始维度: (100000, 2048)
x_train 的塑形维度: (100000, 2, 32, 32)


In [20]:
classical_nn = ClassicalNN(channels=CHANNELS, img_height=IMG_HEIGHT, com_height=COM_HEIGHT)
hybrid_nn = HybridNN(classical_nn=classical_nn)

In [21]:
hybrid_nn(x_test[0:1])  # 测试前向传播是否成功

TypeError: conv2d() received an invalid combination of arguments - got (numpy.ndarray, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, tuple of ints padding = 0, tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, str padding = "valid", tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
